In [ ]:
import RingDataset
from Models import CNNModel, CNNModelWide, CNNModelDeep, RNNGenerator, Distiller, MLP, RNNModel, QGRU2
import Models
import os
import numpy as np
import pickle
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import re
import time

In [ ]:
testset =  RingDataset.RingDataset('core4ToSlice3_test.pkl', threshold=42)

testloader = DataLoader(testset, batch_size=256, num_workers=4)

studentdim = 8
gen=QGRU2(42, scale=0.25, dim=studentdim, drop=0.0)
assert os.path.isfile('./gans/best_{}_{}.pth'.format('qgru', studentdim))
gen.load_state_dict(torch.load('./gans/best_{}_{}.pth'.format('qgru', studentdim)))


def quantizer(arr, std=8):
    return torch.round(arr*std)/std

testset =  RingDataset.RingDataset('core4ToSlice3_test.pkl', threshold=42)
valset =  RingDataset.RingDataset('core4ToSlice3_valid.pkl', threshold=42)

testloader = DataLoader(testset, batch_size=128, num_workers=4)
valloader = DataLoader(valset, batch_size=128, num_workers=4)


In [ ]:
def shifter(arr, window=32):
    dup = arr[:,None,:].expand(arr.size(0), arr.size(1)+1, arr.size(1))
    dup2 = dup.reshape(arr.size(0), arr.size(1), arr.size(1)+1)
    shifted = dup2[:,:window,:-window]
    return shifted

In [ ]:
model = gen
model.eval()

model_int8 = torch.quantization.quantize_dynamic(
    model, {nn.GRUCell, nn.Linear}, dtype=torch.qint8
)
print(model_int8)

input_fp32, _ = next(iter(testloader))
shifted = shifter(input_fp32)

'''
model.qconfig = torch.quantization.get_default_qconfig('fbgemm')

# Prepare the model for static quantization. This inserts observers in
# the model that will observe activation tensors during calibration.
model_fp32_prepared = torch.quantization.prepare(model)

# calibrate the prepared model to determine quantization parameters for activations
# in a real world setting, the calibration would be done with a representative dataset

model_fp32_prepared(shifted, hidden_fp32)

# Convert the observed model to a quantized model. This does several things:
# quantizes the weights, computes and stores the scale and bias value to be
# used with each activation tensor, and replaces key operators with quantized
# implementations.
model_int8 = torch.quantization.convert(model_fp32_prepared)

# run the model, relevant calculations will happen in int8
'''

halfstudent = model_int8

# run the model, relevant calculations will happen in int8
#res = model_int8(shifted)
#res2 = model(shifted)
#print(res-res2)

In [ ]:
cooldown = 100
#halfstudent.eval()
for _ in range(3):
    #classifier_test = CNNModel(42, dim=256).cuda()
    classifier_test = MLP(42, dim=256).cuda()
    criterion = nn.CrossEntropyLoss()

    lastacc = 0.0
    lastnorm = 0.0
    optim_c2 = torch.optim.Adam(classifier_test.parameters(), lr=1e-4)
    for e in range(cooldown):
        classifier_test.train()
        for x,y in valloader:
            xdata, ydata = x.cuda(), y.cuda()
            shifted = shifter(xdata.cpu())
            #train classifier
            optim_c2.zero_grad()
            perturb = halfstudent(shifted).view(shifted.size(0),-1).cuda()
            #perturb = gen(xdata[:,31:])
            #interleaving?
            output = classifier_test(xdata[:,31:]+perturb.detach().float())
            loss_c = criterion(output, ydata)
            loss_c.backward()
            optim_c2.step()


        mloss = 0.0
        totcorrect = 0
        totcount = 0
        mnorm = 0.0
        zerocorrect = 0
        zerocount = 0
        onecorrect = 0
        onecount = 0
        #evaluate classifier

        with torch.no_grad():
            classifier_test.eval()
            for x,y in testloader:
                xdata, ydata = x.cuda(), y.cuda()
                shifted = shifter(xdata.cpu())
                perturb = halfstudent(shifted).view(shifted.size(0),-1).cuda()
                perturb = quantizer(perturb)
                #perturb = gen(xdata[:,31:])
                norm = torch.mean(perturb)
                output = classifier_test(xdata[:,31:]+perturb.float())
                loss_c = criterion(output, ydata)
                pred = output.argmax(axis=-1)
                mnorm += norm.item()/len(testloader)
                mloss += loss_c.item()/len(testloader)
                #macc += ((pred==ydata).sum().float()/pred.nelement()).item()/len(testloader)
                totcorrect += (pred==ydata).sum().item()
                totcount += y.size(0)
                zerocorrect += ((pred==0)*(ydata==0)).sum().item()
                zerocount += (ydata==0).sum().item()
                onecorrect += ((pred==1)*(ydata==1)).sum().item()
                onecount += (ydata==1).sum().item()
            macc = float(totcorrect)/totcount
            zacc = float(zerocorrect)/zerocount
            oacc = float(onecorrect)/onecount
            if (e+1)%10 == 0:
                print("epoch {} \t zacc {:.6f}\t oneacc {:.6f}\t loss {:.6f}\t Avg perturb {:.6f}\n".format(e+1, zacc, oacc, mloss, mnorm))
            if cooldown - e <= 10:
                lastacc += macc/10
                lastnorm += mnorm/10
            
    print("Last 10 acc: {:.6f}\t perturb: {:.6f}".format(lastacc,lastnorm))



In [ ]:
train_x = []
train_y = []
test_x = []
test_y = []
perturb_x = []
with torch.no_grad():
    for x,y in valloader:
        shifted = shifter(x)
        #train classifier
        perturb = halfstudent(shifted).view(shifted.size(0),-1)
        perturbed_x = x[:,31:]+perturb
        for p in perturb:
            perturb_x.append(torch.round(p*8).int().numpy())
        for p in perturbed_x:
            train_x.append(p.numpy())
        for y_i in y:
            train_y.append(y_i.item())

    for x,y in testloader:
        shifted = shifter(x)
        #train classifier
        perturb = halfstudent(shifted).view(shifted.size(0),-1)
        perturbed_x = x[:,31:]+perturb
        for p in perturbed_x:
            test_x.append(p.numpy())
        for y_i in y:
            test_y.append(y_i.item())
print(len(train_y))

In [ ]:
from sklearn import svm
clf = svm.SVC(gamma='auto')
clf.fit(train_x, train_y)
pred_y = clf.predict(test_x)
#tpred_y = clf.predict(train_x)
(pred_y == test_y).sum()/len(pred_y)
#(tpred_y == train_y).sum()/len(train_y)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
clf2 = KNeighborsClassifier(n_neighbors=25)
clf2.fit(train_x, train_y)
pred_y = clf2.predict(test_x)
#tpred_y = clf.predict(train_x)
(pred_y == test_y).sum()/len(pred_y)
#(tpred_y == train_y).sum()/len(train_y)

In [ ]:
#clf.support_vectors_.shape
clf.dual_coef_.shape
#clf.intercept_
#p = clf.get_params()

In [ ]:
for x,y in valloader:
    shifted = shifter(x)
    #train classifier
    break

traced_encoder = torch.jit.trace(halfstudent.encoder, shifted[0:1,:,0])
hs = halfstudent.encoder(shifted[0:1,:,0])
hs_h = torch.zeros_like(hs)
traced_gru = torch.jit.trace(halfstudent.gru1, (hs, hs_h))
print(traced_encoder.code)
print(traced_gru.code)
traced_decoder = torch.jit.trace(halfstudent.decoder, hs)
print(traced_decoder.code)
traced_encoder.save('quantized_encoder.pt')
traced_gru.save('quantized_gru.pt')
traced_decoder.save('quantized_decoder.pt')

In [ ]:
class ToScript(nn.Module):
    def __init__(self, encoder, gru, decoder):
        super().__init__()
        self.encoder = encoder
        self.gru = gru
        self.decoder = decoder
    def forward(self, x, h):
        y = self.gru(self.encoder(x), h)
        return self.decoder(y) , y
toTrace = ToScript(halfstudent.encoder, halfstudent.gru1, halfstudent.decoder)
traced_model = torch.jit.trace(toTrace, (torch.randn(1,32), torch.zeros(1,8)))
traced_model.save('quantized_model.pt')

In [ ]:
traced_encoder(torch.rand(1,32))

In [ ]:
gen2 = Models.OffsetGenerator(42, scale=6.0)
offset_x = []
with torch.no_grad():
    for x,y in valloader:
            shifted = shifter(x)
            #train classifier
            perturb = gen2(shifted).view(shifted.size(0),-1)
            for p in perturb:
                offset_x.append(torch.round(p*8).int().numpy())
            
#torch.tensor(offset_x).mean()

In [ ]:
offset_x[20:30]

In [ ]:
with open('perturb_trace.log', 'w') as pfile:
    for p in perturb_x[:100]:
        for l in p:
            pfile.write(str(l)+'\n')
with open('offset_trace.log', 'w') as pfile:
    for p in offset_x[:100]:
        for l in p:
            pfile.write(str(l)+'\n')

In [ ]:
import math
def capacity(oacc, zacc):
    omarginal = 0.5*(oacc + 1 - zacc)
    zmarginal = 0.5*(1+zacc-oacc)
    psum = oacc*math.log2(oacc/omarginal)
    psum += (1-oacc)*math.log2((1-oacc)/zmarginal)
    psum += zacc*math.log2(zacc/zmarginal)
    psum += (1-zacc)*math.log2((1-zacc)/omarginal)
    return 0.5*psum
capacity(0.52,0.52)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
randloader = DataLoader(testset, batch_size=4, shuffle=True)
loaditer = iter(randloader)
x,y = next(loaditer)
shifted = shifter(x)
perturb = halfstudent(shifted).view(shifted.size(0),-1)
original = x[:,31:]
added = original+perturb.detach()
fig = plt.figure(figsize=(16,4))
for idx in range(1,5):
    ax1 = fig.add_subplot(1,4,idx)
    ax1.set_title(str(y[idx-1].item()))
    xaxis = np.arange(1,43)
    yaxis = original[idx-1]*testset.std + testset.med
    ax1.plot(xaxis,yaxis)
    yaxis2 = added[idx-1]*testset.std + testset.med
    ax1.plot(xaxis,yaxis2)
    axes = plt.gca()
    #print(perturb[idx-1]*8)
    print(perturb[idx-1].mean().item()*testset.std)
    #axes.set_ylim([145,220])
plt.savefig('fig_perturb.pdf', format='pdf')

In [ ]:
from Models import GaussianGenerator, GaussianSinusoid, OffsetGenerator
randloader = DataLoader(testset, batch_size=3, shuffle=True)
loaditer = iter(randloader)
x,y = next(loaditer)
shifted = shifter(x)
gau = nn.Sequential(GaussianGenerator(42, 10), nn.ReLU())
sin = nn.Sequential(GaussianGenerator(42, 10), nn.ReLU())
pad = OffsetGenerator(42, 5)
gp = gau(shifted)
sp = sin(shifted)
pp = pad(shifted)
perturb = pp
pp[0] = gp[0]
#pp[1] = sp[1]
original = x[:,31:]
added = original+perturb.detach()
names = ['Gaussian', 'Constant Pad']
fig = plt.figure(figsize=(8,4))
for idx in range(1,3):
    ax1 = fig.add_subplot(1,2,idx)
    ax1.set_title(names[idx-1])
    xaxis = np.arange(1,43)
    yaxis = original[idx-1]*testset.std + testset.med
    ax1.plot(xaxis,yaxis)
    yaxis2 = added[idx-1]*testset.std + testset.med
    ax1.plot(xaxis,yaxis2)
    axes = plt.gca()
    #print(perturb[idx-1]*8)
    print(perturb[idx-1].mean().item()*testset.std)
    #axes.set_ylim([145,220])
plt.savefig('fig_noise.pdf', format='pdf')

In [ ]:
testset.std

In [ ]:
from Models import RNNGenerator2
cooldown = 100
#halfstudent.eval()
teacher = RNNGenerator2(42, scale=0.25, dim=128, drop=0.0).cuda()
#teacher.load_state_dict(torch.load('./gans/best_adv_160.pth'))
teacher.load_state_dict(torch.load('./models/best_adv_128.pth'))

for _ in range(2):
    classifier_test = CNNModelWide(42, dim=256).cuda()
    #classifier_test = RNNModel(42, dim=256).cuda()
    criterion = nn.CrossEntropyLoss()

    lastacc = 0.0
    lastnorm = 0.0
    optim_c2 = torch.optim.Adam(classifier_test.parameters(), lr=1e-4)
    for e in range(cooldown):
        classifier_test.train()
        for x,y in valloader:
            xdata, ydata = x.cuda(), y.cuda()
            shifted = shifter(xdata)
            #train classifier
            optim_c2.zero_grad()
            perturb = teacher(shifted).view(shifted.size(0),-1)
            #perturb = gen(xdata[:,31:])
            #interleaving?
            output = classifier_test(xdata[:,31:]+perturb.detach().float())
            loss_c = criterion(output, ydata)
            loss_c.backward()
            optim_c2.step()


        mloss = 0.0
        totcorrect = 0
        totcount = 0
        mnorm = 0.0
        zerocorrect = 0
        zerocount = 0
        onecorrect = 0
        onecount = 0
        #evaluate classifier

        with torch.no_grad():
            classifier_test.eval()
            for x,y in testloader:
                xdata, ydata = x.cuda(), y.cuda()
                shifted = shifter(xdata)
                perturb = teacher(shifted).view(shifted.size(0),-1)
                perturb = quantizer(perturb)
                #perturb = gen(xdata[:,31:])
                norm = torch.mean(perturb)
                output = classifier_test(xdata[:,31:]+perturb.float())
                loss_c = criterion(output, ydata)
                pred = output.argmax(axis=-1)
                mnorm += norm.item()/len(testloader)
                mloss += loss_c.item()/len(testloader)
                #macc += ((pred==ydata).sum().float()/pred.nelement()).item()/len(testloader)
                totcorrect += (pred==ydata).sum().item()
                totcount += y.size(0)
                zerocorrect += ((pred==0)*(ydata==0)).sum().item()
                zerocount += (ydata==0).sum().item()
                onecorrect += ((pred==1)*(ydata==1)).sum().item()
                onecount += (ydata==1).sum().item()
            macc = float(totcorrect)/totcount
            zacc = float(zerocorrect)/zerocount
            oacc = float(onecorrect)/onecount
            if (e+1)%10 == 0:
                print("epoch {} \t zacc {:.6f}\t oneacc {:.6f}\t loss {:.6f}\t Avg perturb {:.6f}\n".format(e+1, zacc, oacc, mloss, mnorm))
            if cooldown - e <= 10:
                lastacc += macc/10
                lastnorm += mnorm/10
            
    print("Last 10 acc: {:.6f}\t perturb: {:.6f}".format(lastacc,lastnorm))



In [ ]:
train_x = []
train_y = []
test_x = []
test_y = []
perturb_x = []
with torch.no_grad():
    for x,y in valloader:
        shifted = shifter(x)
        #train classifier
        perturb = teacher(shifted.cuda()).view(shifted.size(0),-1)
        perturbed_x = (x[:,31:]+perturb.cpu()).cpu()
        for p in perturbed_x:
            train_x.append(p.numpy())
        for y_i in y:
            train_y.append(y_i.item())

    for x,y in testloader:
        shifted = shifter(x)
        #train classifier
        perturb = teacher(shifted.cuda()).view(shifted.size(0),-1)
        perturbed_x = (x[:,31:]+perturb.cpu()).cpu()
        for p in perturbed_x:
            test_x.append(p.numpy())
        for y_i in y:
            test_y.append(y_i.item())
print(len(train_y))

In [ ]:
from sklearn import svm
clf = svm.SVC(gamma='auto')
clf.fit(train_x, train_y)
pred_y = clf.predict(test_x)
#tpred_y = clf.predict(train_x)
(pred_y == test_y).sum()/len(pred_y)
#(tpred_y == train_y).sum()/len(train_y)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
clf2 = KNeighborsClassifier(n_neighbors=25)
clf2.fit(train_x, train_y)
pred_y = clf2.predict(test_x)
#tpred_y = clf.predict(train_x)
(pred_y == test_y).sum()/len(pred_y)
#(tpred_y == train_y).sum()/len(train_y)